## Basic terminologies and questions!

"Curiousity is the key to find answers to the most terrible problems"



However, in this example our example is not that terrible! The power of neural networks has been proved time and again and state of the art innovations in pattern recognition, computer vision, speech recognition etc. have paved ways for better understanding into how a machine actually thinks! Days are still far away when computers would think like we do but the good better part is, the quest has begun! With the emergence of high computational power and improvements in distributed computing that equips us to play with even bigger data with less compute has been tremendous. 2017 it is and probably this is the perfect time to get your hands dirty in the really 'deep' world of neural networks!

To start with tensorflow, instead of going for a MNIST or an IRIS dataset, let's try to understand how the neural nets work , firstly, without tensorflow or any libraries and then with the most popular framework that rules, Tensorflow!





In [18]:
# Intro to tensorflow using toy data set

import numpy as np
import random
import tensorflow as tf

def feature_set(test_size):
    #print("All imported perfectly!")

    # after importing the necessary dependencies or packages, we are good to go with our toy dataset!

    # This would be quite intuitive than the 4-bit comparator and hence I guess we could train a model 
    #that predicts stuff really well

    features = []
    features.append([[0,0,0,0],[0,1]])
    features.append([[0,0,0,1],[0,1]])
    features.append([[0,0,1,0],[0,1]])
    features.append([[0,0,1,1],[0,1]])
    features.append([[0,1,0,0],[0,1]])
    features.append([[0,1,0,1],[0,1]])
    features.append([[0,1,1,0],[0,1]])
    features.append([[0,1,1,1],[0,1]])
    features.append([[1,0,0,0],[0,1]])
    features.append([[1,0,0,1],[0,1]])
    features.append([[1,0,1,0],[0,1]])
    features.append([[1,0,1,1],[1,0]])
    features.append([[1,1,0,0],[0,1]])
    features.append([[1,1,0,1],[1,0]])
    features.append([[1,1,1,0],[0,1]])
    features.append([[1,1,1,1],[1,0]])

    #you can see the output is dependent upon the first and last bits in the input, when both of them are 1 its 1 else the 
    #answer is 0, lets see whether tensorflow is powerful enough to crack this!

    #print(features)

    #now lets convert this to the np array but lets shuffle it first

    random.shuffle(features)
    features = np.array(features)

    #print(features)
    
    #fixing testing size, i.e., how much data do you split for testing and training purposes!
    testing_size = int(len(features)*test_size)
    
    train_x = list(features[:,0][:-testing_size])
    test_x = list(features[:,0][-testing_size:])
    
    train_y = list(features[:,1][:-testing_size])
    test_y=list(features[:,1][-testing_size:])
    
    #print(len(train_x))
    #print(len(test_x))
    
    return train_x,test_x,train_y,test_y
    
    

#feature_set(0.3)
#feature_set(0.4)




#as our feature set is ready lets begin with our tensorflow model

#asign the returned value to respective variables of train and test

train_x, test_x, train_y, test_y = feature_set(0.3)

#let define the no. of nodes of our hidden layers:

n_nodes_h11 = 40
n_nodes_h12 = 40

#classes in our output

n_classes = 2

#no. of iterations and batch size:

hm_epochs = 2000
batch_size = 4

x = tf.placeholder('float')
y = tf.placeholder('float')

# random weights and biases

hidden_1_layer = {'f_fum':n_nodes_h11,
                 'weight': tf.Variable(tf.random_normal([len(train_x[0]),n_nodes_h11])),
                 'bias': tf.Variable(tf.random_normal([n_nodes_h11]))}


hidden_2_layer = {'f_fum':n_nodes_h12,
                 'weight': tf.Variable(tf.random_normal([n_nodes_h11,n_nodes_h12])),
                 'bias': tf.Variable(tf.random_normal([n_nodes_h12]))}


output_layer = {'f_fum':None,
                 'weight': tf.Variable(tf.random_normal([n_nodes_h12,n_classes])),
                 'bias': tf.Variable(tf.random_normal([n_classes]))}






def neural_network_model(data):
    
    #hidden layer general :  (data * W) + bias
    
    #hidden layer 1
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']),hidden_1_layer['bias'])
    l1 = tf.sigmoid(l1)
    
    # hidden layer 2, now data is the output of the first hidden layer
    
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.sigmoid(l2)
    
    # output layer, now i/p is l2
    
    output = tf.add(tf.matmul(l2,output_layer['weight']),output_layer['bias'])
    
    return output


#traing our model

def train_neural(x):
    
    #use model definition
    
    prediction = neural_network_model(x)
    
    #cost fucntion
    
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

    # optimize for cost using gradient descent, here's the key thing!
    
    optimizer = tf.train.GradientDescentOptimizer(1).minimize(cost)
    
    #TF session
    
    with tf.Session() as sess:
        
        #initializing variables
        
        sess.run(tf.global_variables_initializer())
        
        #loop through specified no. of iterations
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            
            #handle batch sized chunks of training data
            
            while i<len(train_x):
                start = i
                end = i + batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                # the black box begins
                
                _, c =sess.run([optimizer,cost], feed_dict={x: batch_x,y:batch_y})
                
                epoch_loss += c
                i+=batch_size
                last_cost = c
                
            if(epoch%(hm_epochs/5))==0:
                print('Epoch',epoch, 'completed out of',hm_epochs,'Cost:', last_cost)
                
        
        #print accuracy
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy:', accuracy.eval({x:test_x, y:test_y}))
        
        #print predictions
        
        for i,t in enumerate(test_x):
            print('prediction for :', test_x[i])
            output = prediction.eval(feed_dict = {x: [test_x[i]]})
            # normalise the predictui values:
            print(tf.sigmoid(output[0][0]).eval(), tf.sigmoid(output[0][1]).eval())
            
train_neural(x)
            











Epoch 0 completed out of 2000 Cost: 6.79488e-06
Epoch 400 completed out of 2000 Cost: 0.000229276
Epoch 800 completed out of 2000 Cost: 0.000113623
Epoch 1200 completed out of 2000 Cost: 7.46492e-05
Epoch 1600 completed out of 2000 Cost: 5.51909e-05
Accuracy: 0.75
prediction for : [0, 0, 0, 1]
0.000300691 0.999658
prediction for : [1, 1, 0, 1]
0.177402 0.874036
prediction for : [0, 1, 0, 1]
0.000200887 0.998954
prediction for : [0, 0, 1, 0]
0.000116789 0.999905
